# STAC Zarr 

Invoke a Common Workflow Language CommandLineTool to generate the STAC Zarr

This notebook is linked to: https://eoap.github.io/zarr-cloud-native-format/cwl-cli/stac-zarr



## Setup

In [1]:
export WORKSPACE=/workspace/zarr-cloud-native-format
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

curl -q -L https://github.com/eoap/zarr-cloud-native-format/releases/download/0.5.0/app-water-bodies.0.5.0.cwl > ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl 2> /dev/null

## Run the STAC Zarr generation CommandLineTool

Inspect and use `cwltool` to run the STAC Zarr generation definition:


In [2]:
cat ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl | yq e '.["$graph"][7]' -

class: CommandLineTool
id: stac-zarr
requirements:
  InlineJavascriptRequirement: {}
  ResourceRequirement:
    coresMax: 1
    ramMax: 512
hints:
  DockerRequirement:
    dockerPull: ghcr.io/eoap/zarr-cloud-native-format/stac-zarr@sha256:459a06153db8fffb2fc5b672bb9caf96ba689c7e731000c65e9ba047b0644a69
baseCommand: ["stac-zarr"]
arguments: []
inputs:
  stac_catalog:
    type: Directory
    inputBinding:
      prefix: --stac-catalog
outputs:
  zarr_stac_catalog:
    outputBinding:
      glob: .
    type: Directory


Run the CWL description, but first prepare the parameters.

The previous step generated the water bodies detection geotif:

In [3]:
cat stac-generation-results.json 

{
    "temp_stac_catalog": {
        "location": "file:///workspace/zarr-cloud-native-format/runs/hicmgmcc",
        "basename": "hicmgmcc",
        "class": "Directory",
        "listing": [
            {
                "class": "Directory",
                "location": "file:///workspace/zarr-cloud-native-format/runs/hicmgmcc/water-bodies",
                "basename": "water-bodies",
                "listing": [
                    {
                        "class": "File",
                        "location": "file:///workspace/zarr-cloud-native-format/runs/hicmgmcc/water-bodies/collection.json",
                        "basename": "collection.json",
                        "size": 1351,
                        "checksum": "sha1$299a72c299d255d14b7655e76824d9119a72cc6e",
                        "path": "/workspace/zarr-cloud-native-format/runs/hicmgmcc/water-bodies/collection.json"
                    },
                    {
                        "class": "Directory",
            

Let's build the job parameters file

In [4]:
cat <<EOF > stac-zarr-generation-params.yaml
stac_catalog: 
  class: Directory
  path: $( cat stac-generation-results.json | jq -r '.temp_stac_catalog.path' )
EOF

cat stac-zarr-generation-params.yaml | yq .

stac_catalog:
  class: Directory
  path: /workspace/zarr-cloud-native-format/runs/hicmgmcc


In [5]:


cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl#stac-zarr \
    stac-zarr-generation-params.yaml > stac-zarr-generation-results.json 2> stac-zarr-generation.log

Let's look at the content of the stderr:

In [6]:
cat stac-zarr-generation.log | egrep -v "WARNING|JSHINT"

INFO /home/fbrito/.local/bin/cwltool 3.1.20250110105449
INFO Resolved '/workspace/zarr-cloud-native-format/cwl-workflow/app-water-bodies.cwl#stac-zarr' to 'file:///workspace/zarr-cloud-native-format/cwl-workflow/app-water-bodies.cwl#stac-zarr'
Error: no such object: "ghcr.io/eoap/zarr-cloud-native-format/stac-zarr@sha256:459a06153db8fffb2fc5b672bb9caf96ba689c7e731000c65e9ba047b0644a69"
INFO ['podman', 'pull', 'ghcr.io/eoap/zarr-cloud-native-format/stac-zarr@sha256:459a06153db8fffb2fc5b672bb9caf96ba689c7e731000c65e9ba047b0644a69']
Trying to pull ghcr.io/eoap/zarr-cloud-native-format/stac-zarr@sha256:459a06153db8fffb2fc5b672bb9caf96ba689c7e731000c65e9ba047b0644a69...
Getting image source signatures
Copying blob sha256:1d93c12cf2b0c0ba2b2892f57feb93e5c351daba20fb5f23f83cb3e2b9019630
Copying blob sha256:8ec988941d6694de13ed8cb1505c0eb38bf3777bab0acc157ff18974d7350470
Copying blob sha256:ce6203c8c201f1d0c673adf51ba833952b0f23bcb3a27022803769de34192c15
Copying blob sha256:5b65ed2d7cbcac8f414

Let's inspect the stdout produced. 

In [7]:
cat stac-zarr-generation-results.json | jq . -

{
  "zarr_stac_catalog": {
    "location": "file:///workspace/zarr-cloud-native-format/runs/lq_o_dwl",
    "basename": "lq_o_dwl",
    "class": "Directory",
    "listing": [
      {
        "class": "Directory",
        "location": "file:///workspace/zarr-cloud-native-format/runs/lq_o_dwl/water-bodies",
        "basename": "water-bodies",
        "listing": [
          {
            "class": "Directory",
            "location": "file:///workspace/zarr-cloud-native-format/runs/lq_o_dwl/water-bodies/water-bodies.zarr",
            "basename": "water-bodies.zarr",
            "listing": [
              {
                "class": "File",
                "location": "file:///workspace/zarr-cloud-native-format/runs/lq_o_dwl/water-bodies/water-bodies.zarr/zarr.json",
                "basename": "zarr.json",
                "size": 66,
                "checksum": "sha1$cf1d977edd01e274f9a606c34616fed99ca5e601",
                "path": "/workspace/zarr-cloud-native-format/runs/lq_o_dwl/water-bo